## AutoLiftout XL EVAL



In [ ]:
# openfibsem-baseline-34.pt
# autolamella-02-34.pt


%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import matplotlib.pyplot as plt

TEST_DATA_PATH = "/home/patrick/github/data/liftout/train-new/test"

TEST_DATA_PATH = "/home/patrick/github/data/liftout/train-new/dm-embryo"


filenames = glob.glob(os.path.join(TEST_DATA_PATH, "*.tif"))


print(len(filenames))

In [ ]:
from fibsem.segmentation.model import load_model
from fibsem.detection import detection
from fibsem.structures import FibsemImage

CHECKPOINT_BASELINE = "openfibsem-baseline-34.pt" # autoliftout-base
CHECKPOINT_ONE = "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/xl/base/nc3/earlystop/model.pt"
CHECKPOINT_v3 = "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/xl/finetune/nc3/earlystop/model.pt"
CHECKPOINT_v4 = "/home/patrick/github/fibsem/fibsem/segmentation/models/autoliftout/med/fintune/model.pt"


CHECKPOINTS = [CHECKPOINT_BASELINE, CHECKPOINT_ONE, CHECKPOINT_v3, CHECKPOINT_v4]
ENCODERS = ["resnet34", "resnet50", "resnet50", "resnet34"]
N_CLASSES = [3, 3, 3, 3]

FEATURES = [detection.NeedleTip(), detection.LamellaCentre()]

DETECTIONS = []

i=0
for fname in filenames:
    print(i, fname)
    img = FibsemImage.load(fname)

    dets: list[detection.DetectedFeatures] = []

    for CHECKPOINT, ENCODER, N_C in zip(CHECKPOINTS, ENCODERS, N_CLASSES):
        print(CHECKPOINT)
        model = load_model(CHECKPOINT, encoder=ENCODER, nc=N_C)
        
        det = detection.detect_features(img, model, 
                                            features=FEATURES,
                                            pixelsize=25e-09, filter=False)
        dets.append(det)
    
    DETECTIONS.append(dets)

    i+=1
    if i == 15:
        break
    print('-'*80)

In [ ]:


for fname, dets in zip(filenames, DETECTIONS):

    titles = [f"{checkpoint}" for checkpoint in CHECKPOINTS]
    titles[-3:] = ["autoliftout-xl-base-50-early-stop", "autoliftout-xl-finetune-50-early-stop", "autoliftout-xl-finetune-34"]

    fig = detection.plot_detections(dets, titles=titles)

    # save figure
    # plot_fname = os.path.basename(fname).replace(".tif", ".png")
    
    # fig.savefig(plot_fname, dpi=300, bbox_inches="tight")

In [ ]:
# dataset sizes

DATA_PATH_BASE  = "/home/patrick/github/data/liftout/train-new"
BASELINE_TRAIN_PATH = "/home/patrick/github/data/liftout/training/train/images"

PATH_TRAIN_2 = os.path.join(DATA_PATH_BASE, "train")
PATH_TRAIN_3 = os.path.join(DATA_PATH_BASE, "train3")

files_train_2 = glob.glob(os.path.join(PATH_TRAIN_2, "*.tif"))
files_train_3 = glob.glob(os.path.join(PATH_TRAIN_3, "*.tif"))
baseline_train = glob.glob(os.path.join(BASELINE_TRAIN_PATH, "*.tif"))

print("autolamella dataset sizes")
print("baseline: ", len(baseline_train))
print("train2: ", len(files_train_2))
print("train3: ", len(files_train_3))


import plotly.express as px

# plot count on top of bars
fig = px.bar(x=["baseline (common)", "train2", "train3"], 
    y=[len(baseline_train), len(files_train_2), len(files_train_3)], 
    text=[len(baseline_train), len(files_train_2), len(files_train_3)],
    color=["baseline (common)", "train2", "train3"],
    title="AutoLamella Training Dataset Sizes")

fig.show()
